In [1]:
import xpress as xp
import pandas as pd
import numpy as np

In [2]:
Orders = pd.read_excel("OrderList.xlsx", header=6, index_col=0)
Orders

,Position 1,Position 2,Position 3,Position 4,Position 5
Order No.,,,,,
1,50,30,0,0,0
2,49,18,76,0,0
3,72,52,51,41,35
4,50,4,0,0,0
5,76,19,26,80,6
...,...,...,...,...,...
1996,60,46,35,0,0
1997,8,43,70,77,31
1998,46,0,0,0,0


In [3]:
a = Orders.iloc[0]
print(a)

Position 1    50
Position 2    30
Position 3     0
Position 4     0
Position 5     0
Name: 1, dtype: int64


In [4]:
Allocation = pd.read_csv("CurrentAllocation (table).csv", index_col=0, header=3)
Allocation

,Product Group
Shelf,
1,45
2,79
3,39
4,68
5,73
...,...
92,0
93,0
94,0


In [5]:
Allocation._get_value(1, 'Product Group')

45

In [6]:
DistanceMatrix = pd.read_excel("DistanceMatrix.xlsx", sheet_name="DistanceMatrix Squares", index_col=0, header=2)
DistanceMatrix

,1,2,3,4,5,6,7,8,9,10,...,88,89,90,91,92,93,94,95,96,Packaging
1,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,2
2,1,0,1,2,3,4,5,6,7,8,...,21,22,23,24,25,26,27,28,27,3
3,2,1,0,1,2,3,4,5,6,7,...,22,23,24,25,26,27,28,27,26,4
4,3,2,1,0,1,2,3,4,5,6,...,23,24,25,26,27,28,27,26,25,5
5,4,3,2,1,0,1,2,3,4,5,...,24,25,26,27,28,27,26,25,24,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,25,26,27,28,27,26,25,24,23,22,...,5,4,3,2,1,0,1,2,3,23
94,26,27,28,27,26,25,24,23,22,21,...,6,5,4,3,2,1,0,1,2,24
95,27,28,27,26,25,24,23,22,21,20,...,7,6,5,4,3,2,1,0,1,25
96,28,27,26,25,24,23,22,21,20,19,...,8,7,6,5,4,3,2,1,0,26


In [7]:
print(type(DistanceMatrix))
DistanceMatrix.at["Packaging",50]

<class 'pandas.core.frame.DataFrame'>


4

In [8]:
next_shelf = Allocation.index[Allocation['Product Group'] == 50].tolist()
print(type(next_shelf))
next_shelf

<class 'list'>


[79]

# Distance function in default order (squares)

In [9]:
distance = 0
for i in range(2000):
    print("order mnumber: ", i)
    order = Orders.iloc[i]
    print("current order is:",order)
    current_shelf = "Packaging"
    for j in range(5):
        print("current shelf is:", current_shelf)
        if order[j] != 0:
            print("current good is:", order[j])
            next_shelves = Allocation.index[Allocation['Product Group'] == order[j]].tolist()
            print("corresponding shelves is: ", next_shelves)
            current_distances = {}
            for k in next_shelves:
                print(k)
                current_distances[k] = DistanceMatrix.at[current_shelf, k]
                print("corresponding distance is: ",current_distances)
            next_shelf = min(current_distances, key=current_distances.get)
            current_distance = current_distances[next_shelf]
            print("next shelf is: ",next_shelf)
            print("corresponding distance is: ",current_distance)
            distance += current_distance
        current_shelf = next_shelf
    distance += DistanceMatrix.at[current_shelf, "Packaging"]
    print("total distance is: ",distance)

order mnumber:  0
current order is: Position 1    50
Position 2    30
Position 3     0
Position 4     0
Position 5     0
Name: 1, dtype: int64
current shelf is: Packaging
current good is: 50
corresponding shelves is:  [79]
79
corresponding distance is:  {79: 9}
next shelf is:  79
corresponding distance is:  9
current shelf is: 79
current good is: 30
corresponding shelves is:  [51]
51
corresponding distance is:  {51: 4}
next shelf is:  51
corresponding distance is:  4
current shelf is: 51
current shelf is: 51
current shelf is: 51
total distance is:  18
order mnumber:  1
current order is: Position 1    49
Position 2    18
Position 3    76
Position 4     0
Position 5     0
Name: 2, dtype: int64
current shelf is: Packaging
current good is: 49
corresponding shelves is:  [73]
73
corresponding distance is:  {73: 3}
next shelf is:  73
corresponding distance is:  3
current shelf is: 73
current good is: 18
corresponding shelves is:  [62]
62
corresponding distance is:  {62: 13}
next shelf is:  62

/tmp/ipykernel_2183/2067949568.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if order[j] != 0:
/tmp/ipykernel_2183/2067949568.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("current good is:", order[j])
/tmp/ipykernel_2183/2067949568.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  next_shelves = Allocation.index[Allocation['Product Group'] == order[j]].tolist()



corresponding distance is:  4
current shelf is: 13
current good is: 15
corresponding shelves is:  [85]
85
corresponding distance is:  {85: 27}
next shelf is:  85
corresponding distance is:  27
current shelf is: 85
current good is: 67
corresponding shelves is:  [16]
16
corresponding distance is:  {16: 24}
next shelf is:  16
corresponding distance is:  24
current shelf is: 16
total distance is:  2842
order mnumber:  47
current order is: Position 1    34
Position 2    60
Position 3     0
Position 4     0
Position 5     0
Name: 48, dtype: int64
current shelf is: Packaging
current good is: 34
corresponding shelves is:  [24]
24
corresponding distance is:  {24: 25}
next shelf is:  24
corresponding distance is:  25
current shelf is: 24
current good is: 60
corresponding shelves is:  [15]
15
corresponding distance is:  {15: 9}
next shelf is:  15
corresponding distance is:  9
current shelf is: 15
current shelf is: 15
current shelf is: 15
total distance is:  2892
order mnumber:  48
current order 

total distance is:  18022
order mnumber:  288
current order is: Position 1    74
Position 2    22
Position 3    30
Position 4     0
Position 5     0
Name: 289, dtype: int64
current shelf is: Packaging
current good is: 74
corresponding shelves is:  [74]
74
corresponding distance is:  {74: 4}
next shelf is:  74
corresponding distance is:  4
current shelf is: 74
current good is: 22
corresponding shelves is:  [61]
61
corresponding distance is:  {61: 11}
next shelf is:  61
corresponding distance is:  11
current shelf is: 61
current good is: 30
corresponding shelves is:  [51]
51
corresponding distance is:  {51: 10}
next shelf is:  51
corresponding distance is:  10
current shelf is: 51
current shelf is: 51
total distance is:  18052
order mnumber:  289
current order is: Position 1    28
Position 2    50
Position 3    30
Position 4    67
Position 5    18
Name: 290, dtype: int64
current shelf is: Packaging
current good is: 28
corresponding shelves is:  [65]
65
corresponding distance is:  {65: 19

# Distance function in Greedy (square)

In [10]:
def dis_fun(allo):
    distance = 0
    for i in range(2000):
        # print("order mnumber: ", i)
        order = Orders.iloc[i]
        # print("current order is:",order)
        num_goods = np.count_nonzero(order)
        # print("the number of goods in this order: ", num_goods )
        current_shelf = "Packaging"

        visited_goods = []  # 使用list跟踪访问过的物品编号
        
        while len(visited_goods) < num_goods:
            within_order_distances = {}
            for j in range(5):
                # print("current shelf is:", current_shelf)
                # print("checking good is:", order[j])
            
                if order[j] != 0 and order[j] not in visited_goods:
                    # print("current good is:", order[j])
                    within_good_shelves = allo.index[allo['Product Group'] == order[j]].tolist() # 目前的物品对应的多个货架
                    # print("corresponding shelves for current good is: ", within_good_shelves)

                    # 对目前的GOOD对应的每个货架对应的距离， 找出最小距离的货架
                    within_good_distances = {} 
                    for k in within_good_shelves:
                        # print(k)
                        # within_good_distances[k] = DistanceMatrix.at[current_shelf, k]
                        within_good_distances[k] = DistanceMatrix[current_shelf][k]
                        # print("corresponding distance is: ",within_good_distances)
                    least_within_good_shelf = min(within_good_distances, key=within_good_distances.get)
                    least_within_good_distance = within_good_distances[least_within_good_shelf]
                    # print("chosen shelf for current good is: ",least_within_good_shelf)
                    # print("corresponding distance for the chosen shelf is: ",least_within_good_distance)

                    # 将最小距离的货架和对应的距离存入within_order_distances
                    within_order_distances[least_within_good_shelf] = least_within_good_distance
                    # print("within_order_distances: ", within_order_distances)

            # print("the final within order distance for current shelf is: ", within_order_distances)
            least_within_order_shelf = min(within_order_distances, key=within_order_distances.get)
            # print("the shelf with the least distance for this shelf is: ", least_within_order_shelf)
            next_shelf = least_within_order_shelf
            # print("thus the next shelf is: ", next_shelf)
            least_within_order_distance = within_order_distances[least_within_order_shelf]
            # print("the correlated distance is: ", least_within_order_distance)
            next_good = allo._get_value(next_shelf, "Product Group")
            # print("the next good to pick is: ", next_good)
            visited_goods.append(next_good)
            # print("current visited goods are: ", visited_goods)
            current_shelf = next_shelf

            distance += least_within_order_distance
            # print("the accumulative distance is: ", distance)
            # next_good =  allo.index[allo['Shelf'] == next_shelf]
            # next_good = allo._get_value(next_shelf, "Product Group")
            # current_distance = within_order_distances[next_shelf]
            # visited_goods.append(next_good)
            # print("visited goods are:", visited_goods)

        distance += DistanceMatrix.at[current_shelf, "Packaging"]
        # print("the accumulative distance after this order is: ", distance)
        
    return(distance)

## Allocate products randomly

In [25]:
def allocation_gen():
    # Create a DataFrame with integers from 1 to 90
    df = pd.DataFrame({'Product Group': range(1, 91)})

    # Shuffle the order of integers randomly
    df = df.sample(frac=1).reset_index(drop=True)

    # Generate 6 random unique integers from 1 to 90
    additional_integers = np.random.choice(np.arange(1, 91), size=6, replace=False)

    # Create a DataFrame for additional integers
    additional_df = pd.DataFrame({'Product Group': additional_integers})

    # Concatenate the additional integers to the existing DataFrame
    df = pd.concat([df, additional_df], ignore_index=True)

    df.index = np.arange(1, 97)
    df.index.name = 'shelf'

    return(df)

In [26]:
Allo_init = allocation_gen()
distance_init = dis_fun(allo=Allo_init)
for i in range(20):
    Allocation = allocation_gen()
    distance = dis_fun(allo=Allocation)
    if distance <= distance_init:
        distance_init = distance
        Allocation = Allo_init
print(Allo_init)
print(distance_init)


/tmp/ipykernel_2183/363952484.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if order[j] != 0 and order[j] not in visited_goods:
/tmp/ipykernel_2183/363952484.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  within_good_shelves = allo.index[allo['Product Group'] == order[j]].tolist() # 目前的物品对应的多个货架


       Product Group
shelf               
1                 62
2                 57
3                 75
4                  4
5                 52
...              ...
92                17
93                33
94                53
95                54
96                25

[96 rows x 1 columns]
90514


## Local Search Heuristics 

In [23]:
Allo_init

,Product Group
shelf,
1,76
2,54
3,67
4,89
5,19
...,...
92,76
93,84
94,46


In [24]:

for i in range(95):
    Allo_init = Allo_init
    Distance_init = dis_fun(allo=Allo_init)

    # Swap values at specific positions (e.g., swap values at index 0 and index j)
    index1 = i
    index2 = i + 1

    # Create a new DataFrame to store the swapped values
    Allo_next = Allo_init.copy()

    # Get the values at the specified indices
    value1 = Allo_init.iloc[index1, 0]
    value2 = Allo_init.iloc[index2, 0]

    # Swap the values
    Allo_next.iloc[index1, 0] = value2
    Allo_next.iloc[index2, 0] = value1

    # Find the allocation and distance 
    Allo_next = Allo_next
    Distance_next = dis_fun(allo=Allo_next)

    if Distance_next <= Distance_init:
        Distance_init = Distance_next
        Allo_init = Allo_next
print(Allo_init)
print(Distance_init)

/tmp/ipykernel_2183/363952484.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if order[j] != 0 and order[j] not in visited_goods:
/tmp/ipykernel_2183/363952484.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  within_good_shelves = allo.index[allo['Product Group'] == order[j]].tolist() # 目前的物品对应的多个货架


       Product Group
shelf               
1                 76
2                 67
3                 89
4                 19
5                 45
...              ...
92                84
93                46
94                24
95                14
96                76

[96 rows x 1 columns]
90632


In [27]:

for i in range(95):
    Allo_init = Allocation
    Distance_init = dis_fun(allo=Allo_init)

    # Swap values at specific positions (e.g., swap values at index 0 and index j)
    index1 = i
    index2 = i + 1

    # Create a new DataFrame to store the swapped values
    Allo_next = Allo_init.copy()

    # Get the values at the specified indices
    value1 = Allo_init.iloc[index1, 0]
    value2 = Allo_init.iloc[index2, 0]

    # Swap the values
    Allo_next.iloc[index1, 0] = value2
    Allo_next.iloc[index2, 0] = value1

    # Find the allocation and distance 
    Allo_next = Allo_next
    Distance_next = dis_fun(allo=Allo_next)

    if Distance_next <= Distance_init:
        Distance_init = Distance_next
        Allo_init = Allo_next
print(Allo_init)
print(Distance_init)

/tmp/ipykernel_2183/363952484.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if order[j] != 0 and order[j] not in visited_goods:
/tmp/ipykernel_2183/363952484.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  within_good_shelves = allo.index[allo['Product Group'] == order[j]].tolist() # 目前的物品对应的多个货架


       Product Group
shelf               
1                  7
2                 76
3                 19
4                 83
5                 75
...              ...
92                45
93                81
94                34
95                64
96                85

[96 rows x 1 columns]
91974


In [13]:
number_of_positions = 5
Positions = range(number_of_positions)

In [15]:
prob_order = xp.problem(name='each_order')

go = np.array([xp.var( name = 'go_{0}_{1}'.format(i+1,j+1))
                    for i in Positions for j in Positions], dtype=xp.npvar).reshape(number_of_positions,number_of_positions)

print(go)

prob_order.addVariable(go)

[[go_1_1 go_1_2 go_1_3 go_1_4 go_1_5]
 [go_2_1 go_2_2 go_2_3 go_2_4 go_2_5]
 [go_3_1 go_3_2 go_3_3 go_3_4 go_3_5]
 [go_4_1 go_4_2 go_4_3 go_4_4 go_4_5]
 [go_5_1 go_5_2 go_5_3 go_5_4 go_5_5]]
